In [1]:
import numpy as np
import os
import pandas as pd
import soundfile as sf

import matplotlib.pyplot as plt
import matplotlib.ticker as tick
from matplotlib import colors
from matplotlib.patches import Rectangle

import scipy.io.wavfile as wav
import datetime as dt
from pathlib import Path

from torch import multiprocessing
from tqdm import tqdm

In [2]:
import sys

# append the path of the
# parent directory
sys.path.append('..')
sys.path.append('../src/')
sys.path.append('../src/models/bat_call_detector/batdetect2/')

In [3]:
from src.cfg import get_config
from src.pipeline import pipeline
import src.batdt2_pipeline as batdetect2_pipeline

In [4]:
dirpath = f"{Path.home()}/Documents/UBNA/Symposium/Recordings/recover-20230605/UBNA_001"

recover_folder = dirpath.split('/')[-2]
recover_date = recover_folder.split('-')[1]
audiomoth_folder = dirpath.split('/')[-1]
audiomoth_unit = audiomoth_folder.split('_')[-1]

save_dir = f'../output_dir/{recover_folder}/{audiomoth_folder}'
detfile_name = f'batdetect2_pipeline__{recover_folder}_{audiomoth_folder}'

[dirpath, save_dir, detfile_name]

['/Users/adityakrishna/Documents/UBNA/Symposium/Recordings/recover-20230605/UBNA_001',
 '../output_dir/recover-20230605/UBNA_001',
 'batdetect2_pipeline__recover-20230605_UBNA_001']

In [11]:
## Each file in directory has own set of detections from 0 to file_length
## Audio file corresponding to any detection is stored in .csv output's 'input_file' column

# batdetect2_pipeline.run_pipeline(dirpath, f"{detfile_name}.csv", save_dir, f'../output/tmp')

In [6]:
field_records = batdetect2_pipeline.get_field_records(Path("ubna_2022b.csv"))
site_name = batdetect2_pipeline.get_site_name(field_records, recover_date, audiomoth_unit)
# batdetect2_pipeline.plot_dets_as_activity_grid(dirpath, detfile_name, save_dir, site_name)

In [11]:
recover_folder = dirpath.split('/')[-2]
audiomoth_folder = dirpath.split('/')[-1]
dets = pd.read_csv(f'{save_dir}/{detfile_name}.csv')
activity = np.array([])
activity_times = []
activity_dates = []

files = np.unique(dets['input_file'].values)
files

array(['20230531_043000.WAV', '20230531_050000.WAV',
       '20230531_053000.WAV', '20230531_060000.WAV',
       '20230531_063000.WAV', '20230531_070000.WAV',
       '20230531_073000.WAV', '20230531_080000.WAV',
       '20230531_083000.WAV', '20230531_090000.WAV',
       '20230531_093000.WAV', '20230531_100000.WAV',
       '20230531_103000.WAV', '20230531_110000.WAV',
       '20230531_113000.WAV', '20230601_043000.WAV',
       '20230601_050000.WAV', '20230601_053000.WAV',
       '20230601_060000.WAV', '20230601_063000.WAV',
       '20230601_070000.WAV', '20230601_073000.WAV',
       '20230601_080000.WAV', '20230601_083000.WAV',
       '20230601_090000.WAV', '20230601_093000.WAV',
       '20230601_100000.WAV', '20230601_103000.WAV',
       '20230601_110000.WAV', '20230601_113000.WAV',
       '20230602_043000.WAV', '20230602_050000.WAV',
       '20230602_053000.WAV', '20230602_060000.WAV',
       '20230602_063000.WAV', '20230602_070000.WAV',
       '20230602_073000.WAV', '20230602_080000

In [17]:
for file in batdetect2_pipeline.get_files_from_dir(dirpath):
    filedets = dets.loc[dets['input_file']==file]
    activity = np.hstack([activity, len(filedets)])

    file_dt = dt.datetime.strptime(file, "%Y%m%d_%H%M%S.WAV")

    file_time = dt.datetime.strftime(file_dt, "%H:%M")
    if (not(activity_times.__contains__(file_time))):
        activity_times.append(file_time)

    file_date = dt.datetime.strftime(file_dt, "%m/%d/%y")
    if (not(activity_dates.__contains__(file_date))):
        activity_dates.append(file_date)
        activity_for_date = np.array([])

activity_times

FileNotFoundError: [Errno 2] No such file or directory: '/Users/adityakrishna/Documents/UBNA/Symposium/Recordings/recover-20230605/UBNA_001'

In [ ]:
activity = activity.reshape((len(activity_dates), len(activity_times))).T

plt.rcParams.update({'font.size': 16})
plt.figure(figsize=(12, 8))
plt.title(f"Activity from {site_name}", loc='left', y=1.05)
plt.imshow(activity+1, norm=colors.LogNorm(vmin=1, vmax=10e3))
plt.yticks(np.arange(0, len(activity_times), 2)-0.5, activity_times[::2])
plt.xticks(np.arange(0, len(activity_dates))-0.5, activity_dates, rotation=60)
plt.colorbar()
if save:
    plt.savefig(f"{output_dir}/activity_{recover_folder}_{audiomoth_folder}.png", bbox_inches='tight')
plt.tight_layout()
plt.show()